In [ ]:
import pandas as pd
import numpy as np

energy_data = pd.read_csv("Extra//energy.csv")
energy_data.timestamp = pd.to_datetime(energy_data.timestamp, format='%Y-%m-%d %H:%M:%S')
energy_data.index = energy_data.timestamp
resampled_energy = energy_data.resample("5s").mean()

env_data = pd.read_csv("Extra//environment.csv")
env_data.timestamp = pd.to_datetime(env_data.timestamp, format='%Y-%m-%d %H:%M:%S')
env_data.index = env_data.timestamp
resampled_env = env_data.resample("5s").mean()

In [ ]:
resampled_energy['reactive_power'] = resampled_energy[["Reactive Power A average [kVAr]","Reactive Power B average [kVAr]","Reactive Power C average [kVAr]"]].mean(axis=1)
resampled_energy['thdi'] = resampled_energy[["THDI A average [%]","THDI B average [%]","THDI C average [%]"]].mean(axis=1)
resampled_energy['thdu'] = resampled_energy[["THDU A average [%]","THDU B average [%]","THDU C average [%]"]].mean(axis=1)
resampled_energy['current'] = resampled_energy[["Current A average [A]","Current B average [A]","Current C average [A]"]].mean(axis=1)
resampled_energy['power_factor'] = resampled_energy[["Power Factor A average","Power Factor B average","Power Factor C average"]].mean(axis=1)
useful_data = resampled_energy.join(resampled_env)
useful_data = useful_data[["reactive_power", "thdi", "thdu", "current","Xacc", "pitch","yaw","sound"]]
useful_data = useful_data.dropna()
useful_data = useful_data.iloc[:-(useful_data.shape[0] % 10)]
useful_data

In [ ]:
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()

scaled_data = standard_scaler.fit_transform(useful_data)
reshaped_scaled_data = scaled_data.reshape(scaled_data.shape[0]//10, 10, scaled_data.shape[1])
reshaped_scaled_data.shape

In [ ]:
from keras.layers import Input, LSTM, RepeatVector
from keras.models import Model


input_shape = (reshaped_scaled_data.shape[1], reshaped_scaled_data.shape[2])
encoding_dim = 64
inputs = Input(shape=input_shape)
encoded = LSTM(encoding_dim, activation='relu')(inputs)

decoded = RepeatVector(input_shape[0])(encoded)
decoded = LSTM(input_shape[1], activation='relu', return_sequences=True)(decoded)

autoencoder = Model(inputs, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.fit(reshaped_scaled_data, reshaped_scaled_data, epochs=32, batch_size=64)
predictions = autoencoder.predict(reshaped_scaled_data)
predictions = predictions.reshape(predictions.shape[0]*predictions.shape[1], predictions.shape[2])

In [ ]:
mse = np.square(np.subtract(scaled_data, predictions))
anomalies = np.where(mse > 500)
anomalies

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
plot_df = pd.DataFrame(scaler.fit_transform(useful_data.values), columns=useful_data.columns, index=useful_data.index)

for i in list(zip(anomalies[0],anomalies[1]))[:10]:
    anomaly = i[0]
    new_df = plot_df.loc[str(useful_data.iloc[anomaly-100].name):str(useful_data.iloc[anomaly+100].name)]
    new_df.insert(loc=0,column='Date', value=pd.to_datetime(new_df.index))

    fig = px.line(new_df, x = 'Date', y = useful_data.columns[i[1]])
    fig = go.Figure(data=fig.data).update_layout(xaxis_title="Time", yaxis_title=useful_data.columns[i[1]])

    fig.add_vrect(x0=str(useful_data.iloc[anomaly-5].name),x1=str(useful_data.iloc[anomaly-4].name),fillcolor="black", opacity=1)
    fig.add_vrect(x0=str(useful_data.iloc[anomaly+5].name),x1=str(useful_data.iloc[anomaly+6].name),fillcolor="black", opacity=1)
    fig.add_vrect(x0=str(useful_data.iloc[anomaly].name),x1=str(useful_data.iloc[anomaly+1].name),fillcolor="black", opacity=0.2)

    fig.show()

In [ ]:
"""
import plotly.express as px
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
plot_df = pd.DataFrame(scaler.fit_transform(useful_data.values), columns=useful_data.columns, index=useful_data.index)

for anomaly in anomalies[0]:
    new_df = plot_df.loc[str(useful_data.iloc[anomaly-100].name):str(useful_data.iloc[anomaly+100].name)]
    new_df.insert(loc=0,column='Date', value=pd.to_datetime(new_df.index))

    fig = px.line(new_df, x = 'Date', y = useful_data.columns)
    fig = go.Figure(data=fig.data).update_layout(xaxis_title="Time", yaxis_title="Data")

    fig.add_vrect(x0=str(useful_data.iloc[anomaly-5].name),x1=str(useful_data.iloc[anomaly-4].name),fillcolor="black", opacity=1)
    fig.add_vrect(x0=str(useful_data.iloc[anomaly+5].name),x1=str(useful_data.iloc[anomaly+6].name),fillcolor="black", opacity=1)
    fig.add_vrect(x0=str(useful_data.iloc[anomaly].name),x1=str(useful_data.iloc[anomaly+1].name),fillcolor="black", opacity=0.2)

    fig.show()
"""